In [1]:
import os
import cv2
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.layers import Lambda,Dense,ReLU,MaxPooling2D,Flatten,Add
from tensorflow.keras.layers import Dense,Input,MaxPool2D,Conv2D,Dropout,Concatenate
import matplotlib.pyplot as plt
from utils import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [2]:
num_classes = 10
input_shape = (28, 28, 1)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
inputs = Input(shape=(28, 28, 1))

output_1 = Involution(channel=3, group_number=1, kernel_size=3, stride=1, reduction_ratio=2, name="involution_1")(inputs)
output_1 = ReLU()(output_1)
output_1 = Concatenate()([output_1, inputs])

output_1 = Involution(channel=3, group_number=1, kernel_size=3, stride=1, reduction_ratio=2, name="involution_2")(inputs)
output_1 = ReLU()(output_1)

output = Concatenate()([output_1, inputs])

output = Flatten()(output)
output = Dropout(0.5)(output)
output = Dense(num_classes, activation="softmax")(output)
model = Model(inputs=inputs, outputs=output)

model.summary()
# keras.utils.plot_model(model, to_file='model.png')


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
involution_2 (Involution)       (None, 28, 28, 1)    24          input_2[0][0]                    
__________________________________________________________________________________________________
re_lu_3 (ReLU)                  (None, 28, 28, 1)    0           involution_2[0][0]               
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 28, 28, 2)    0           re_lu_3[0][0]                    
                                                                 input_2[0][0]         

In [5]:
batch_size = 128
epochs = 20
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
422/422 [==============================] - 4s 10ms/step - loss: 0.6796 - accuracy: 0.7941 - val_loss: 0.2871 - val_accuracy: 0.9197
Epoch 2/20
422/422 [==============================] - 4s 8ms/step - loss: 0.4173 - accuracy: 0.8794 - val_loss: 0.2558 - val_accuracy: 0.9362
Epoch 3/20
422/422 [==============================] - 4s 8ms/step - loss: 0.3866 - accuracy: 0.8890 - val_loss: 0.2412 - val_accuracy: 0.9348
Epoch 4/20
422/422 [==============================] - 4s 8ms/step - loss: 0.3626 - accuracy: 0.8960 - val_loss: 0.2302 - val_accuracy: 0.9358
Epoch 5/20
422/422 [==============================] - 4s 9ms/step - loss: 0.3470 - accuracy: 0.8983 - val_loss: 0.2133 - val_accuracy: 0.9437
Epoch 6/20
422/422 [==============================] - 4s 9ms/step - loss: 0.3262 - accuracy: 0.9045 - val_loss: 0.1988 - val_accuracy: 0.9462
Epoch 7/20
422/422 [==============================] - 4s 9ms/step - loss: 0.3050 - accuracy: 0.9094 - val_loss: 0.1789 - val_accuracy: 0.9522
Epoch

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
score

[0.14661769568920135, 0.9598000049591064]